In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
arr = np.load("./samples/pose_est.npy")

In [ ]:
plt.plot(arr[:,0], arr[:,1])

In [ ]:
VOXEL_SZ = 0.2
DEBUG=False

In [ ]:
# process_input = from_polar if args.no_polar else lambda x : x
# print 

def pose2matrix(translation_list, rotation_angle_list, zoom_list=[1,1,1]):
    # trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom_vec = np.array(zoom_list)
    # transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom_vec)
    transform_mat = transforms3d.affines.compose(translation_list, rot_mat, zoom_list)
    return transform_mat

# Function to transform given lidar pcd to ground truth to get it upright
def transform_lidar_to_gt_frame(pcd):
    new_pcd = copy.deepcopy(pcd)
    transformation_lidar2gt = pose2matrix([0,0,0], [0,0,90],[1,1,-1])
    new_pcd.transform(transformation_lidar2gt)
    return new_pcd

# Function to get pcd for given range image in torch.cuda
def get_pcd_from_img(img):
    img = img * 100
    frame = from_polar(img).detach().cpu().numpy()[0]
    # frame_actual = np.array([frame_image[:29] for frame_image in frame])
    frame_flat = frame.reshape((3,-1))
    some_pcd = o3d.PointCloud()
    some_arr = frame_flat.T
    some_pcd.points = o3d.utility.Vector3dVector(some_arr)

    new_some_pcd = transform_lidar_to_gt_frame(some_pcd)
    return new_some_pcd

# Function to get ICP pose for given src pcd and dst pcd
def get_icp_pose(src, dst, voxel_size=VOXEL_SZ):
    def crop_pcd(old_pcd, crop_min_arr=np.array([-100,-100,-2]), crop_max_arr=np.array([100,100,100])):
        np.random.seed(0)
        pcd = copy.deepcopy(old_pcd)

        cropped_pcd = o3d.geometry.crop_point_cloud(pcd, crop_min_arr, crop_max_arr)
        pcd = cropped_pcd
        return pcd

    def prepare_dataset(source, target, voxel_size):
        def preprocess_point_cloud(pcd, voxel_size):
            pcd_down = o3d.geometry.voxel_down_sample(pcd, voxel_size)
            radius_normal = voxel_size * 2
            o3d.geometry.estimate_normals(
                pcd_down,
                o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
            radius_feature = voxel_size * 5
            pcd_fpfh = o3d.registration.compute_fpfh_feature(
                pcd_down,
                o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
            return pcd_down, pcd_fpfh
        source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
        target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
        return source_down, target_down, source_fpfh, target_fpfh

    def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
        distance_threshold = voxel_size * 1.5
        if DEBUG:
            print("start execute global reg")
        result = o3d.registration.registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
            o3d.registration.TransformationEstimationPointToPoint(False), 4, [
                o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.registration.CorrespondenceCheckerBasedOnDistance(
                    distance_threshold)
            ], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
        if DEBUG:
            print("finish execute global reg")
        return result

    def refine_registration(source, target, voxel_size, trans_init):
        distance_threshold = voxel_size * 0.4
        result = o3d.registration.registration_icp(
                    source, target, distance_threshold, trans_init,
                    o3d.registration.TransformationEstimationPointToPlane())
                    # o3d.registration.TransformationEstimationPointToPlane())
        return result
    # get_icp_pose execution starts here
    if DEBUG:
        print("start icp pose")
    # source = crop_pcd(src)
    source = src
    if DEBUG:
        print("cropped src")
    # target = crop_pcd(dst)
    target = dst
    if DEBUG:
        print("cropped dst")

    source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(source, target, voxel_size)
    if DEBUG:
        print("prepared dataset")
    result_ransac = execute_global_registration(source_down, target_down,
                                                source_fpfh, target_fpfh,
                                                voxel_size)
    if DEBUG:
        print("executed global reg")
    result_icp = refine_registration(source_down, target_down, voxel_size, result_ransac.transformation)
    if DEBUG:
        print("refined reg")

    evaluation = o3d.registration.evaluate_registration(source_down, target_down, voxel_size * 5, result_icp.transformation)
    if DEBUG:
        print("evaluated")

    # print("Before ICP")
    # draw_registration_result(source_down, target_down, pose2matrix([0,0,0], [0,0,0],[1,1,1]))

    # print("After ICP")
    # draw_registration_result(source_down, target_down, result_icp.transformation)

    return result_icp.transformation, evaluation

# Function to give slam pose for given two consecutive range images in torch.cuda
def get_slam_pose_transform(recon_curr_img, recon_next_img):
    dynamic_pcd_curr = get_pcd_from_img(recon_curr_img)
    if DEBUG:
        print("got pcd curr")

    dynamic_pcd_next = get_pcd_from_img(recon_next_img)
    if DEBUG:
        print("got pcd next")

    slam_pose_transform, slam_pose_err = get_icp_pose(dynamic_pcd_curr, dynamic_pcd_next)
    if DEBUG:
        print("got slam pose")

    # print("Before ICP")
    # draw_registration_result(dynamic_pcd_curr, dynamic_pcd_next,
    #                             pose2matrix([0,0,0], [0,0,0],[1,1,1]))

    # print("After ICP")
    # draw_registration_result(dynamic_pcd_curr, dynamic_pcd_next, slam_pose_transform)

    gc.collect()
    return slam_pose_transform, slam_pose_err 

In [ ]:
PCD_PATH = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/dynamic/3/_out"

In [ ]:
import os
import open3d as o3d
from tqdm import tqdm_notebook

In [ ]:
def getint(name):
    return int(name.split('.')[0])

In [ ]:
pose_list = []
old_pt = np.array([0,0,0,1])
pose_list.append(old_pt)

for pcd_idx in tqdm_notebook(range(len(os.listdir(PCD_PATH))-1)):
    src_pcd = o3d.read_point_cloud(os.path.join(PCD_PATH, str(pcd_idx+1)+".ply"))
    dst_pcd = o3d.read_point_cloud(os.path.join(PCD_PATH, str(pcd_idx+2)+".ply"))
    slam_pose_transform, slam_pose_err = get_icp_pose(src_pcd, dst_pcd)
    
    old_pt = pose_list[-1]
    new_pt = np.matmul(slam_pose_transform, old_pt)
    pose_list.append(new_pt)
    break
    

In [ ]:
slam_pose_transform

In [ ]:
def draw_registration_result(src, dst, transformation):
    source = copy.deepcopy(src)
    target = copy.deepcopy(dst)
    
    source.paint_uniform_color([1, 0, 0]) # red
    target.paint_uniform_color([0, 0, 1]) # blue
    target.transform(transformation)
    o3d.visualization.draw_geometries([source, target], width=1280, height=800)

In [ ]:
print("Before ICP")
draw_registration_result(src_pcd, dst_pcd, pose2matrix([0,0,0], [0,0,0],[1,1,1]))

print("After ICP")
draw_registration_result(src_pcd, dst_pcd, result_icp.transformation)

In [ ]:
pose_arr = np.array(pose_list)

In [ ]:
plt.plot(pose_arr[:,0], pose_arr[:,1])

In [ ]:
def crop_pcd(old_pcd, crop_min_arr=np.array([-100,-100,-2]), crop_max_arr=np.array([100,100,100])):
    np.random.seed(0)
    pcd = copy.deepcopy(old_pcd)

    cropped_pcd = o3d.geometry.crop_point_cloud(pcd, crop_min_arr, crop_max_arr)
    pcd = cropped_pcd
    return pcd

In [ ]:
pose_list

In [ ]:
for frame_num in range(dynamic_img_curr.shape[0]):
        # recon_curr_frame = recon_curr[frame_num:frame_num+1, :, :IMAGE_HEIGHT, :]
        # recon_next_frame = recon_next[frame_num:frame_num+1, :, :IMAGE_HEIGHT, :]
    recon_curr_frame = recon_curr[frame_num:frame_num+1, :, :, :]
    recon_next_frame = recon_next[frame_num:frame_num+1, :, :, :]

    # Get SLAM Pose as blackbox
    pose_transform, pose_err = get_slam_pose_transform(recon_curr_frame, recon_next_frame)
    old_pt = pose_list[-1]
    new_pt = np.matmul(pose_transform, old_pt)
    pose_list.append(new_pt)